In [1]:
import pandas as pd
import numpy as np

tweet_df = pd.read_pickle('../data/agg_trans_tweets.pickle.gz', compression='gzip')
tweet_df['i'] = list(range(len(tweet_df)))
tweet_df.set_index('date', inplace=True)
tweet_df

,tweet,author,id,i
date,,,,
2021-04-27 21:45:50,@Alicoh1 @qwosl @kdqd3 @HotlineJosh But strang...,Platonista1,1387161089121701892,0
2021-04-27 21:45:49,RT @DavidPaisley: Not only do we export our tr...,daniellismore,1387161086479241218,1
2021-04-27 21:45:45,RT @vickiringer: NC’s bathroom bill discrimina...,CherieAnne,1387161067000840198,2
2021-04-27 21:45:40,@yelloweycat @Mushr00mSteve @BaddestJeanette A...,encyclopath,1387161048352841728,3
2021-04-27 21:45:40,@thehill This CEO is fired for belittling a gu...,rynob11,1387161046612336641,4
...,...,...,...,...
2021-04-20 08:31:11,@idkwtwhnid @GhostlyWeevil @GayestFesh @bigges...,natleah_,1384424393124761600,303843
2021-04-20 08:30:31,RT @HANNAHLUVSDSGC: north carolina wants to pa...,taoIings,1384424225113624578,303844
2021-04-20 08:30:31,RT @mcclure111: If you live in a country where...,Angstlers1,1384424224715067394,303845


## LDA Performance

How well does LDA perform? First we must tokenize and vectorize all of our tweets.

In [2]:
import sklearn
import gzip
import pickle
import sys
import os
sys.path.append("..")
import utils.tokens as ut

stopwords = ut.build_stopword_set()

def tokenizer(tweet):
    return ut.tokenize_tweet(tweet, stopwords)

def autopickle(path):
    """Helper utility to either load a model from file, or store a newly-trained one."""
    def decorator(func):
        if os.path.exists(path):
            with gzip.open(path, 'rb') as file:
                model = pickle.load(file)
        else:
            model = func()
            with gzip.open(path, 'wb') as file:
                pickle.dump(model, file)
        return model
    return decorator
        
@autopickle('../models/gs-tfidf.pickle.gz')
def vectorizer():
    vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(tokenizer=tokenizer)
    vectorizer.fit(tweet_df['tweet'])
    return vectorizer

[nltk_data] Downloading package stopwords to /home/astrid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/astrid/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Next, we will split our tweets into a few different datasets.

In [3]:
N_HYPER_TUNING = 20_000
N_TEST = 50_000

shuffled = tweet_df.sample(frac=1)
df_hyper_tuning = shuffled.iloc[:N_HYPER_TUNING]
df_training = shuffled.iloc[N_HYPER_TUNING:-N_TEST]
df_test = shuffled.iloc[:-N_TEST]
del shuffled  # save memory

print(f'Hyperparameter Tuning: {len(df_hyper_tuning)}')
print(f'Training: {len(df_training)}')
print(f'Test: {len(df_test)}')

Hyperparameter Tuning: 20000
Training: 233848
Test: 253848


Let's run a LDA on a small sample of the training data first to see what it looks like.

In [4]:
@autopickle('../models/gs-sample-lda.pickle.gz')
def sample_lda():
    lda = sklearn.decomposition.LatentDirichletAllocation(
        n_components=20,
        max_iter=15,
        n_jobs=-1,  # all processors
    )
    lda.fit(vectorizer.transform(df_training['tweet'].sample(50_000)))
    return lda

In [5]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(sample_lda, vectorizer.transform(df_test['tweet']), vectorizer, mds='tsne')

/home/astrid/anaconda3/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
9      -10.878068   10.538728       1        1  20.036675
18     -60.884026  -75.591820       2        1   8.642314
11      -2.336119  -53.289074       3        1   5.752431
16     -61.385319  -12.914634       4        1   4.856237
2      -28.737743  126.296997       5        1   4.794099
15       1.066606 -123.552811       6        1   4.691859
7       45.140720   -7.947869       7        1   4.666568
12    -140.795975  -98.530548       8        1   4.292758
1       61.531681  -83.240562       9        1   4.258695
10     -72.563057 -140.396881      10        1   4.188129
19      15.984491   63.578186      11        1   3.856592
5     -160.851624   68.945145      12        1   3.730294
6       85.961830   51.922935      13        1   3.620954
17     -99.020836  109.936348      14        1   3.497870
3     -176.764130   -8.681684      15        1   3.471840
0     -117.189880  -37.420200      16        1   3.284483
8      108.829918  -22.946203      17        1   3.260388
4       47.588097  121.934814      18        1   3.124133
14    -106.467834   30.176889      19        1   3.003421
13     -50.613804   59.328842      20        1   2.970260, topic_info=            Term         Freq        Total Category  logprob  loglift
35691       mass  1437.000000  1437.000000  Default  30.0000  30.0000
49406   shooting  1427.000000  1427.000000  Default  29.0000  29.0000
49627     signed  1530.000000  1530.000000  Default  28.0000  28.0000
12748       bill  3235.000000  3235.000000  Default  27.0000  27.0000
35228     making  1483.000000  1483.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
44208  protected    69.731055   221.728538  Topic20  -5.8207   2.3597
31776      judge    69.957141   312.593726  Topic20  -5.8174   2.0195
55078       true    65.236883   242.814287  Topic20  -5.8873   2.2022
41925     people    83.419893  8561.950519  Topic20  -5.6414  -1.1147
57284       view    67.947610   545.637181  Topic20  -5.8466   1.4333

[1233 rows x 6 columns], token_table=       Topic      Freq              Term
term                                    
341        7  0.452474        #apr25coup
341       15  0.535909        #apr25coup
344        7  0.316571        #apr27coup
344       15  0.671408        #apr27coup
393        7  0.978898  #aseansupportnug
...      ...       ...               ...
62063     20  0.833774              분노하며
62149     20  0.833774             싸우겠다고
62201     20  0.833774               엘리엇
62252     20  0.833774                위해
62374     20  0.815958                차별

[3790 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 19, 12, 17, 3, 16, 8, 13, 2, 11, 20, 6, 7, 18, 4, 1, 9, 5, 15, 14])

Although the results are not too bad, they can definitely be improved.

### Hyperparameter Tuning

We want to find better hyperparameters for our dataset. One way to do this is with grid search. First, let's see what parameters we have available to us for tuning.

In [6]:
sklearn.decomposition.LatentDirichletAllocation().get_params().keys()

/home/astrid/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


dict_keys(['batch_size', 'doc_topic_prior', 'evaluate_every', 'learning_decay', 'learning_method', 'learning_offset', 'max_doc_update_iter', 'max_iter', 'mean_change_tol', 'n_components', 'n_jobs', 'perp_tol', 'random_state', 'topic_word_prior', 'total_samples', 'verbose'])

Now, we will run our grid search on our small hyperparameter tuning dataset.

In [7]:
@autopickle('../models/gs-gridsearch-lda.pickle.gz')
def grid_search_lda():
    gs = sklearn.model_selection.GridSearchCV(
        estimator=sklearn.decomposition.LatentDirichletAllocation(
            n_jobs=-1,
        ),
        param_grid={
            'n_components': [10, 20, 30], 
            'doc_topic_prior': [1/40, 1/30, 1/20, 1/10],
            'topic_word_prior': [1/40, 1/30, 1/20, 1/10],
        },
        verbose=2,
    )
    vectorized_train = vectorizer.transform(df_hyper_tuning['tweet'])
    gs.fit(vectorized_train)
    return gs

grid_search_lda.best_params_

/home/astrid/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'doc_topic_prior': 0.1, 'n_components': 10, 'topic_word_prior': 0.1}

The first grid search was fairly coarse. Let's try narrowing our grid search range.

In [10]:
@autopickle('../models/gs-gridsearch2-lda.pickle.gz')
def grid_search_lda_2():
    bp = grid_search_lda.best_params_
    gs = sklearn.model_selection.GridSearchCV(
        estimator=sklearn.decomposition.LatentDirichletAllocation(
            n_jobs=-1,
            random_state=1,
        ),
        param_grid={
            'n_components': [bp['n_components'] + dn for dn in [-2, 1, 0, 1, 2]], 
            'doc_topic_prior': [1 / (1 / bp['doc_topic_prior'] + dn) for dn in [-2, 0, 2]],
            'topic_word_prior': [1 / (1 / bp['topic_word_prior'] + dn) for dn in [-2, 0, 2]],
        },
        verbose=2,
    )
    vectorized_train = vectorizer.transform(df_hyper_tuning['tweet'])
    gs.fit(vectorized_train)
    return gs

grid_search_lda_2.best_params_

/home/astrid/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'doc_topic_prior': 0.125, 'n_components': 8, 'topic_word_prior': 0.125}

Finally, we will run the best parameters on the full dataset.

In [11]:
@autopickle('../models/gs-hpt-lda.pickle.gz')
def hp_tuned_lda():
    lda = sklearn.decomposition.LatentDirichletAllocation(
        **grid_search_lda_2.best_params_,
        n_jobs=-1,
    )
    lda.fit(vectorizer.transform(df_training['tweet']))
    return lda

/home/astrid/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
pyLDAvis.sklearn.prepare(hp_tuned_lda, vectorizer.transform(df_training['tweet']), vectorizer, mds='tsne')

/home/astrid/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/astrid/anaconda3/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
5     -108.958252 -208.941483       1        1  31.408961
6      186.487061  -98.103249       2        1  11.288849
7       19.245832  160.858780       3        1  10.339235
3      170.658829   75.445076       4        1   9.798378
1       -1.386411  -35.043739       5        1   9.714306
2     -145.800095  106.944611       6        1   9.410104
4       64.834541 -221.825272       7        1   9.385640
0     -196.906036  -58.990913       8        1   8.654527, topic_info=           Term         Freq        Total Category  logprob  loglift
35691      mass  1878.000000  1878.000000  Default  30.0000  30.0000
49406  shooting  1868.000000  1868.000000  Default  29.0000  29.0000
49627    signed  1804.000000  1804.000000  Default  28.0000  28.0000
12748      bill  4004.000000  4004.000000  Default  27.0000  27.0000
26556      goal  1475.000000  1475.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
45381      real   392.699413   752.504882   Topic8  -5.0799   1.7967
59510     youth   398.931917  1372.147826   Topic8  -5.0641   1.2117
57748        wa   404.012350  2969.147429   Topic8  -5.0514   0.4525
33645       law   381.483958  1153.743942   Topic8  -5.1088   1.3404
41925    people   387.985609  9558.933770   Topic8  -5.0919  -0.7572

[562 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
341        5  0.079281  #apr25coup
341        6  0.618903  #apr25coup
341        7  0.166234  #apr25coup
341        8  0.135545  #apr25coup
344        5  0.297886  #apr27coup
...      ...       ...         ...
59510      5  0.368036       youth
59510      6  0.083810       youth
59510      7  0.050286       youth
59510      8  0.290785       youth
60787      8  0.997700           न

[1332 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 7, 8, 4, 2, 3, 5, 1])

In [ ]:
hp_tuned_lda.perplexity(vectorizer.transform(df_test['tweet']))

In [ ]:
hp_tuned_lda.perplexity(vectorizer.transform(df_training['tweet']))